In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urlparse

import torch
from transformers import BertTokenizer, BertLMHeadModel

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/genresell/opt/anaconda3/envs/day1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/genresell/nltk_data...


In [2]:
urls = [
    'https://www.wsj.com/',
    'https://www.cnn.com/',
    'https://www.nytimes.com/',
    'https://www.theguardian.com/international',
    'https://www.reuters.com/news/world'
]

In [3]:
# Initialize an empty list to store the scraped data
articles = []

In [4]:
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles += soup.find_all('article')

In [5]:
data = []
for article in articles:
    article_url = article.find('a')['href']
    if not article_url.startswith('http'):
        article_url = f'https://{urlparse(url).netloc}{article_url}'

    title = article.find('h3')
    if title:
        title = title.text.strip()

    date = article.find('time')
    if date:
        try:
            date = datetime.strptime(date['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ')
            date = date.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            date = None

    author = article.find('span', class_='c-byline__name')
    if author:
        author = author.text.strip()

    content = ''
    content_url = article_url
    content_response = requests.get(content_url)
    content_soup = BeautifulSoup(content_response.content, 'html.parser')
    paragraphs = content_soup.find_all('p')
    for paragraph in paragraphs:
        content += paragraph.text

    data.append({
        'title': title,
        'date': date,
        'author': author,
        'content': content,
        'url': article_url
    })

In [6]:
df = pd.DataFrame(data)
print(df.tail())

                                                title  date author   
19  Robert Kennedy Jr to make 2024 Democratic pres...  None   None  \
20  Tennessee Republicans likely to expel three De...  None   None   
21  Unstable markets drag Canadian M&A, debt issua...  None   None   
22  Telkom Indonesia to merge broadband arm with T...  None   None   
23  Airline SAS seeks equity bids as part of bankr...  None   None   

                                              content   
19  [1/2] Robert F. Kennedy Jr. and his wife actor...  \
20  [1/3] Rep. Justin Pearson, Rep. Justin Jones, ...   
21  TORONTO, April 6 (Reuters) - Canadian dealmake...   
22  April 6 (Reuters) - PT Telekomunikasi Selular ...   
23  April 5 (Reuters) - Scandinavian airline SAS A...   

                                                  url  
19  https://www.reuters.com/article/usa-election-k...  
20  https://www.reuters.com/article/tennessee-poli...  
21  https://www.reuters.com/article/canada-deals/u...  
22  https://

In [7]:
# load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertLMHeadModel.from_pretrained('bert-base-uncased')

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# define function to preprocess text data for BERT
def preprocess_text(text):
    # tokenize text
    tokenized_text = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=127,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    return tokenized_text

In [9]:
# define function to generate summary using BERT
def generate_summary(text):
    # preprocess text
    tokenized_text = preprocess_text(text)

    # generate summary
    summary_ids = model.generate(
        input_ids=tokenized_text['input_ids'],
        attention_mask=tokenized_text['attention_mask'],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    # decode summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [10]:
# preprocess content and generate summary for each article in dataframe
df['content_preprocessed'] = df['content'].apply(preprocess_text)
df['summary'] = df['content'].apply(generate_summary)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [11]:
# view final dataframe with summary column added
print(df.tail())

                                                title  date author   
19  Robert Kennedy Jr to make 2024 Democratic pres...  None   None  \
20  Tennessee Republicans likely to expel three De...  None   None   
21  Unstable markets drag Canadian M&A, debt issua...  None   None   
22  Telkom Indonesia to merge broadband arm with T...  None   None   
23  Airline SAS seeks equity bids as part of bankr...  None   None   

                                              content   
19  [1/2] Robert F. Kennedy Jr. and his wife actor...  \
20  [1/3] Rep. Justin Pearson, Rep. Justin Jones, ...   
21  TORONTO, April 6 (Reuters) - Canadian dealmake...   
22  April 6 (Reuters) - PT Telekomunikasi Selular ...   
23  April 5 (Reuters) - Scandinavian airline SAS A...   

                                                  url   
19  https://www.reuters.com/article/usa-election-k...  \
20  https://www.reuters.com/article/tennessee-poli...   
21  https://www.reuters.com/article/canada-deals/u...   
22  http

In [12]:
analyzer = SentimentIntensityAnalyzer()
df['sentiment_scores'] = df['summary'].apply(lambda x: analyzer.polarity_scores(x))

In [13]:
def get_sentiment_label(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [14]:
df['sentiment_label'] = df['sentiment_scores'].apply(lambda x: get_sentiment_label(x['compound']))

In [15]:
# view final dataframe with sentiment scores column added
print(df.tail())

                                                title  date author   
19  Robert Kennedy Jr to make 2024 Democratic pres...  None   None  \
20  Tennessee Republicans likely to expel three De...  None   None   
21  Unstable markets drag Canadian M&A, debt issua...  None   None   
22  Telkom Indonesia to merge broadband arm with T...  None   None   
23  Airline SAS seeks equity bids as part of bankr...  None   None   

                                              content   
19  [1/2] Robert F. Kennedy Jr. and his wife actor...  \
20  [1/3] Rep. Justin Pearson, Rep. Justin Jones, ...   
21  TORONTO, April 6 (Reuters) - Canadian dealmake...   
22  April 6 (Reuters) - PT Telekomunikasi Selular ...   
23  April 5 (Reuters) - Scandinavian airline SAS A...   

                                                  url   
19  https://www.reuters.com/article/usa-election-k...  \
20  https://www.reuters.com/article/tennessee-poli...   
21  https://www.reuters.com/article/canada-deals/u...   
22  http